In [1]:
import numpy as np
import time
import datetime as dt
import yfinance as yf
from ib_insync import *
import pandas as pd
import sys
import sched
import datetime
import pandas_ta as ta
import warnings

##### Warnings for invalid values/dividing by zero
warnings.filterwarnings("ignore")

def exit():
    print('{} Now the system will exit '.format(datetime.datetime.now()))
    sys.exit()

class CaptureReturn:
    def __init__(self, func, initial_state):
        self.func = func
        self.state = initial_state

    def call_func(self):
        self.output = self.func(self.state)
        return self.output

def get_stock_info(stock): 
    attempts = 3 
    for attempt in range(1, attempts+1): 
        
        data = yf.download(tickers=stock, period = '2d', interval='2m').round(decimals=3)
        if data.empty:
            print(f'Attempt {attempt} Failed, Dataset Empty.')
            time.sleep(3)
            continue
        else:
            print(f'Downloaded in Attempt {attempt}')
            break      
    return data

def place_order(direction, stock, amount): 
    order = MarketOrder(direction, amount)
    contract = Stock(stock, 'SMART', 'USD')
    trade = ib.placeOrder(contract, order)         
    return trade

def order_status(trade): ################
    if trade.orderStatus.status == 'Filled': ################
        fill = trade.fills[-1] ################
        price = fill.execution.avgPrice ################
        print(f'{fill.time} - {fill.execution.avgPrice}')
        return price ################

def trade_algo(data, buy_signals, sell_signals, buy_price, trigger, stock, amount): ################

    def ema(dataset, price, interval):
        ema_column = ta.ema(dataset[price], length=interval)
        data_ema = dataset.assign(ema=ema_column)
        return data_ema
    
    def atr(dataset, high, low, close, interval):
        average_tr = ta.atr(dataset[high], dataset[low], dataset[close])
        data_atr = dataset.assign(atr = average_tr)
        return data_atr
    
    df_ema_12 = ema(data, 'Close', 12).round(decimals=3)
    df_ema_50 = ema(data, 'Close', 50).round(decimals=3)
    df_atr = atr(data, 'High', 'Low', 'Close', 14).round(decimals=3)
    
    high = data["High"]
    open = data["Open"]
    close = data["Close"]
    low = data["Low"]
    atr = df_atr["atr"]
    ema_12 = df_ema_12["ema"]
    ema_50 = df_ema_50["ema"]
    
    body_candle_1 = close[-3] - open[-3]
    body_candle = close[-2] - open[-2]

    high_stick_hammer = abs(high[-2] - close[-2])
    low_stick_hammer = abs(open[-2] - low[-2])

    high_stick_int = abs(high[-2] - open[-2])
    low_stick_int = abs(close[-2] - low[-2])
                
    ##### Hammer Candlestick patterns    
        
    if low_stick_hammer/abs(body_candle) > 3 and body_candle > 0 \
    and high_stick_hammer/low_stick_hammer < 0.25 and ema_12[-2] > ema_50[-2] and trigger != 1:

        trade = place_order('BUY', stock, amount)
        buy_price = close[-2]
        print("bought at ", close[-2], "at ", data.index[-2], 'time now: ',
                  datetime.datetime.now().strftime("%H:%M:%S"), 'with hammer pattern')
        print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "low stick/body: ", 
              low_stick_hammer/abs(body_candle), "high stick/low stick", high_stick_hammer/low_stick_hammer)
        time.sleep(3) 
        buy_signals.append(close[-2])
        trigger = 1
    
    ##### Int Hammer Candlestick patterns    

    elif low_stick_int/abs(body_candle) > 3 and body_candle < 0 \
    and high_stick_int/low_stick_int < 0.25 and ema_12[-2] > ema_50[-2] and trigger != 1:

        trade = place_order('BUY', stock, amount)
        buy_price = close[-2]
        print("bought at ", close[-2], "at ", data.index[-2], 'time now: ',
                  datetime.datetime.now().strftime("%H:%M:%S"), 'with int hammer pattern')
        print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "low stick/body: ", 
              low_stick_int/abs(body_candle), "high stick/low stick", high_stick_int/low_stick_int)

        time.sleep(3) 
        buy_signals.append(close[-2])
        trigger = 1
        
    ##### Bullish/Bearing Engulfing patterns
        
    elif open[-2] <= close[-3] and close[-2] >= open[-3]\
    and abs(body_candle)/abs(body_candle_1) > 4 and body_candle_1 < 0 and body_candle > 0 \
    and close[-2]/open[-2] > 1.03 and ema_12[-2] > ema_50[-2] and trigger != 1:
        
        trade = place_order('BUY', stock, amount)
        buy_price = close[-2]
        print("bought at ", close[-2], "at ", data.index[-2], 'time now: ', 
                  datetime.datetime.now().strftime("%H:%M:%S"), 'with engulfing pattern')
        print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "body ratios: ", 
              abs(body_candle)/abs(body_candle_1))

        time.sleep(3) 
        buy_signals.append(close[-2])
        trigger = 1
        
    ########## ATR sell off
    
    elif close[-2] > buy_price + 3*atr[-2] and trigger != -1 \
    or close[-2] < buy_price - 3*atr[-2] and trigger != -1:
        
        if trigger == 0:
            print("closing price: ", close[-2], "closing time: ", data.index[-2], "time now ", 
                  datetime.datetime.now().strftime("%H:%M:%S"))
            print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "time now: ",
                  datetime.datetime.now().strftime("%H:%M:%S"))
        
        else:
            trade = place_order('SELL', stock, amount)
            print("sold at: ", close[-2], "closing time: ", data.index[-2], "time now ", 
                  datetime.datetime.now().strftime("%H:%M:%S"))
            
            time.sleep(3)
            sell_signals.append(close[-2])
            trigger = -1
            
    else:
        print("closing price: ", close[-2], "closing time: ", data.index[-2], "time now ", 
              datetime.datetime.now().strftime("%H:%M:%S"))
        print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "time now: ",
                  datetime.datetime.now().strftime("%H:%M:%S"))
        
    return buy_signals, sell_signals, buy_price, trigger

def save_csv_file(buy, sell, stock): ################
    ################
    dataset = pd.DataFrame({'BUY': buy, 'SELL': sell,}, columns=[stock,'BUY','SELL'])
    ################
    myfile = dataset.to_csv('C:/Users/shong/Desktop/trade_logs.csv', mode='a', index=False)
    return myfile

def duration(start_hour,start_min,end_hour,end_min):
    time_1 = datetime.timedelta(hours= start_hour, minutes=start_min, seconds=0)
    time_2 = datetime.timedelta(hours= end_hour, minutes=end_min, seconds=0)
    
    time_interval = (time_2 - time_1).total_seconds() + \
                        datetime.timedelta(days=1).total_seconds()
    time_interval = (time_interval/60)/2
    return time_interval

print("functions initialised")

functions initialised


In [2]:
def main():

    #stock = sys.argv[1] 
    #trade_amount = sys.argv[2]
    
    print("establishing connection...")
    util.startLoop()
    global ib
    ib = IB()
    ib.connect(host='127.0.0.1', port=7497, clientId=1)
    print("connection established")   
    
    stock = 'EAR'
    print("Stocks:", stock)
         
    buy_signals = []
    sell_signals = []
    buy_price = 0
    amount = 300
    minute = 50
    trigger = 0
    
    iterations = int(duration(21, minute, 4, 0)) 
    
    scheduler = sched.scheduler(time.time, time.sleep)

    capture_return = CaptureReturn(get_stock_info, stock)
    
    time_now = time.time()

    while True:
        for count in range(iterations-1):
            scheduler.enterabs(time_now + 120*(count+1), 1, capture_return.call_func)
            data = capture_return.call_func()
            
            if data.empty:
                print("Dataset not found at ", datetime.datetime.now().strftime("%H:%M:%S"))
                continue
            
            buy_signals, sell_signals, buy_price, trigger = trade_algo(data, buy_signals, 
                                                            sell_signals, buy_price, trigger, stock, amount)
            print("This is the current count: ", count)
            scheduler.run()

        if trigger == 1:
            time.sleep(80) ##################
            
            place_order('SELL', stock, amount)
            print("sold :) at market close ", data['Close'][-1])
            
            time.sleep(3) #############
            sell_signals.append(data['Close'][-1])
            time.sleep(3)
            file = save_csv_file(buy_signals, sell_signals, stock)
            time.sleep(3)
            ib.disconnect()
            quit(0)
            time.sleep(3)
            exit()
        else:
            file = save_csv_file(buy_signals, sell_signals, stock)
            time.sleep(3)
            ib.disconnect()
            quit(0)
            time.sleep(3)
            exit()

if __name__ == "__main__":
    main()

establishing connection...
connection established
Stocks: EAR
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.953 closing time:  2022-08-17 09:48:00-04:00 time now  21:50:19
open:  3.039 high:  3.09 low:  2.93 time now:  21:50:19
This is the current count:  0
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.95 closing time:  2022-08-17 09:50:00-04:00 time now  21:52:19
open:  2.952 high:  3.03 low:  2.93 time now:  21:52:19
This is the current count:  1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.875 closing time:  2022-08-17 09:52:00-04:00 time now  21:54:19
open:  2.949 high:  2.949 low:  2.825 time now:  21:54:1

[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.601 closing time:  2022-08-17 10:34:00-04:00 time now  22:36:19
open:  2.66 high:  2.68 low:  2.6 time now:  22:36:19
This is the current count:  23
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.589 closing time:  2022-08-17 10:36:00-04:00 time now  22:38:19
open:  2.61 high:  2.62 low:  2.566 time now:  22:38:19
This is the current count:  24
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.605 closing time:  2022-08-17 10:38:00-04:00 time now  22:40:19
open:  2.59 high:  2.61 

[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.881 closing time:  2022-08-17 11:20:00-04:00 time now  23:22:19
open:  2.91 high:  2.91 low:  2.85 time now:  23:22:19
This is the current count:  46
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.9 closing time:  2022-08-17 11:22:00-04:00 time now  23:24:19
open:  2.89 high:  2.91 low:  2.88 time now:  23:24:19
This is the current count:  47
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  2.98 closing time:  2022-08-17 11:24:00-04:00 time now  23:26:19
open:  2.898 high:  2.98 low:  2.87 time now:  23:26:19
This is the current count:  48
[*********************100%*******

[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  3.12 closing time:  2022-08-17 12:06:00-04:00 time now  00:08:19
open:  3.12 high:  3.16 low:  3.1 time now:  00:08:19
This is the current count:  69
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  3.11 closing time:  2022-08-17 12:08:00-04:00 time now  00:10:19
open:  3.13 high:  3.15 low:  3.07 time now:  00:10:19
This is the current count:  70
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  3.14 closing time:  2022-08-17 12:10:00-04:00 time now  00:12:19
open:  3.11 high:  3.15 low:  3.09 time now:  00:12:19
This is the current count:  71
[*********************100%*********

[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  3.07 closing time:  2022-08-17 12:54:00-04:00 time now  00:56:19
open:  3.069 high:  3.08 low:  3.03 time now:  00:56:19
This is the current count:  93
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  3.05 closing time:  2022-08-17 12:56:00-04:00 time now  00:58:19
open:  3.064 high:  3.07 low:  3.03 time now:  00:58:19
This is the current count:  94
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  3.009 closing time:  2022-08-17 12:58:00-04:00 time now  01:00:19
open:  3.05 high:  3.06 

...

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 3 Failed, Dataset Empty.
Dataset not found at  01:23:52
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 1 Failed, Dataset Empty.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 2 Failed, Dataset Empty.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 3 Failed, Dataset Empty.
Dataset not found at  01:24:13
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 1 Failed, Dataset Empty.
[**

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 3 Failed, Dataset Empty.
Dataset not found at  01:28:30
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 1 Failed, Dataset Empty.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 2 Failed, Dataset Empty.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 3 Failed, Dataset Empty.
Dataset not found at  01:28:50
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 1 Failed, Dataset Empty.
[**

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 3 Failed, Dataset Empty.
Dataset not found at  01:33:33
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 1 Failed, Dataset Empty.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 2 Failed, Dataset Empty.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 3 Failed, Dataset Empty.
Dataset not found at  01:33:50
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 1 Failed, Dataset Empty.
[**

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 3 Failed, Dataset Empty.
Dataset not found at  01:37:59
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 1 Failed, Dataset Empty.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 2 Failed, Dataset Empty.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 3 Failed, Dataset Empty.
Dataset not found at  01:38:16
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EAR: No data found for this date range, symbol may be delisted
Attempt 1 Failed, Dataset Empty.
[**

SystemExit: 